In [ ]:
require(EBImage)
require(keras)
require(tidyverse)
require(tidymodels)
require(dplyr)

# system("sudo apt-get -y install libmagick++-dev", intern=TRUE)
# install.packages("magick", verbose=TRUE)
library(magick)

library(magrittr)

In [ ]:
train_images <- read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
ds_train <- read_csv('../input/image-features/ds_train.csv')%>%slice(1:18000)
train_images <- train_images %>% mutate(labels=factor(labels))
ds_train <- ds_train %>% mutate(labels=factor(labels))
head(train_images)
head(ds_train)

In [ ]:
#summary(ds_train)

In [ ]:
#train_images <- train_images %>% slice(1:18000)
#             arrange(desc(labels)) %>% 
#             group_by(labels) %>% slice(1:300)

In [ ]:
labels_count <- ds_train%>% group_by(labels)%>%summarize(n=n())%>% arrange(n)
labels_count

In [ ]:
#summary(train_images)

In [ ]:
train_path='../input/plant-pathology-2021-fgvc8/train_images/'
train <- ds_train$image
train_list <- as.list(train)
train_Count <- length(train_list)
train_Count

In [ ]:
height =20
width  =20

In [ ]:
mage<-image_read("../input/plant-pathology-2021-fgvc8/train_images/80077517781fb94f.jpg")%>%
                image_resize( "300x300")
image_trim(mage)#%>% image_convolve('Sobel')%>% image_negate()
#image_transparent(mage, 'white')

In [ ]:
image<-image_read("../input/plant-pathology-2021-fgvc8/train_images/80077517781fb94f.jpg") %>% 
  image_crop("1200x900+1600+1200")%>%image_resize( "300x225")
#image_border(image_background(image, "hotpink"), "#000080", "20x10")
image
image%>% image_convolve('Sobel')%>% image_negate()
#image_trim(image)
#image <- image_resize(image, "20x20")
#image_noise(image)
#image_negate(image)

In [ ]:
# crop <- function(im, left = 0, top = 0, right = 0, bottom = 0) {
#   d <- dim(im[[1]]); w <- d[2]; h <- d[3]
#   image_crop(im, glue::glue("{w-left-right}x{h-top-bottom}+{left}+{top}"))
# }
# image<- image_read('../input/plant-pathology-2021-fgvc8/train_images/800cbf0ff87721f8.jpg') %>%
#           crop(right = 1200,left=1200, top=600,bottom=600)

In [ ]:
# #train_Count = 20
# mypic_train <- vector('list',train_Count)
# temp_mypic <- vector('list',1)
# for (i in 1:train_Count) 
#     {
#     new_path=paste(train_path, train_list[i],sep="")
#     image <-image_read(new_path) %>% image_crop("1200x900+1600+1200")
#     new_img_path <-image_write(image%>%image_convolve('Sobel')%>% image_negate(),path='new.jpg', format = "jpg")
#     new_image<-image_read(new_img_path)
#     temp_mypic[[1]] <- readImage(new_img_path)
#     temp_mypic[[1]] <- resize(temp_mypic[[1]], height, width)
#     mypic_train[[i]] <- array_reshape(temp_mypic[[1]], c(height*width*3))
# }

mypic_train

In [ ]:
test_path <- '../input/plant-pathology-2021-fgvc8/test_images/'
test <- list.files(path = test_path, pattern = "*.jpg" )
test_list <- as.list(test)
test_count =length(test_list)
test_count

In [ ]:
mypic_test <- vector('list',test_count)
temp_mypic <- vector('list',1)
for (i in 1:test_count) 
    {
    new_path=paste(test_path, test_list[i],sep="")
    image <-image_read(new_path) %>% image_crop("1200x900+1600+1200")
    new_img_path <-image_write(image%>%image_convolve('Sobel')%>% image_negate(),path='new.jpg', format = "jpg")
    new_image<-image_read(new_img_path)
    temp_mypic[[1]] <- readImage(new_img_path)
    temp_mypic[[1]] <- resize(temp_mypic[[1]], height, width)
    mypic_test[[i]] <- array_reshape(temp_mypic[[1]], c(height*width*3))
}

In [ ]:
# mypic_data_train <-as.data.frame(t(as.matrix(as.data.frame(mypic_train))))

# ds_train <- bind_cols(head(train_images,train_Count),mypic_data_train) 
# head(ds_train)

In [ ]:
# ds_train = separate_rows(ds_train,2,sep = " ")
# count(ds_train)
# ds_train <- ds_train%>% mutate(labelsnumber = ifelse(labels == "complex", 1, 
#                                         ifelse(labels == "healthy", 2, 
#                                             ifelse(labels == "rust", 3,
#                                                 ifelse(labels ==  "scab", 4, 
#                                                     ifelse(labels == "frog_eye_leaf_spot", 5, 
#                                                         ifelse(labels ==  "powdery_mildew", 6,99)))))))
# head(ds_train)
# ds_train%>% group_by(labelnumber)%>%summarize(n=n())%>% arrange(n)

In [ ]:
test_mypic_data <-as.data.frame(mypic_test)
test_mypic_data <-as.data.frame(t(as.matrix(test_mypic_data)))

ds_test <- bind_cols(test,test_mypic_data)%>% rename( image = `...1`)
head(ds_test)

In [ ]:
nn_recipe<-recipe(labels~., data=ds_train)%>%
                    step_rm(image)

In [ ]:
nn_model<-mlp(epochs=200, hidden_units=height*width*2, dropout =.1)%>%
          set_mode("classification")%>%
          set_engine("keras", verbose=1) 

In [ ]:
nn_workflow<-workflow()%>%add_recipe(nn_recipe)%>%
                          add_model(nn_model)

In [ ]:
nn_fit<-nn_workflow%>%fit(ds_train)

In [ ]:
nn_fit

In [ ]:
train_results <- predict(nn_fit, new_data = ds_train)%>%bind_cols(ds_train%>%select(labels))

In [ ]:
test_results <- predict(nn_fit, new_data = ds_test)%>%bind_cols(ds_test%>%select(image))
test_results

In [ ]:
metrics(train_results, truth=labels, estimate=.pred_class)

In [ ]:
test_results_sub <- test_results%>%transmute(
    image = image,
    labels = .pred_class %>% as.character()
  )
head(test_results_sub)
write.csv(test_results_sub,"submission.csv",row.names=FALSE)